In [6]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from tqdm import tqdm
from multiprocessing import Pool

from method.functions import *
from method.gradient import numerical_gradient

In [7]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        
        self.params = {}
        self.params["W1"] = weight_init_std*np.random.randn(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        self.params["W2"] = weight_init_std*np.random.randn(hidden_size, output_size)
        self.params["b2"] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params["W1"], self.params["W2"]
        b1, b2 = self.params["b1"], self.params["b2"]
        
        layer1 = np.dot(x, W1) + b1
        hidden1 = sigmoid(layer1)
        layer2 = np.dot(hidden1, W2) + b2
        output = softmax(layer2)
        
        return output
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y,t)

    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t)/float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x,t)
        
        grads = {}
        grads["W1"] = numerical_gradient(loss_W, self.params["W1"])
        grads["b1"] = numerical_gradient(loss_W, self.params["b1"])
        grads["W2"] = numerical_gradient(loss_W, self.params["W2"])
        grads["b2"] = numerical_gradient(loss_W, self.params["b2"])
        
        return grads    

In [8]:
from dataset.mnist import load_mnist

(x_train, t_train),(x_test, t_test) = load_mnist(normalize = True, one_hot_label=True)

In [9]:
train_loss_list = []

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

In [10]:
for i in tqdm(range(iters_num)):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.numerical_gradient(x_batch, t_batch)
    
    for key in ("W1", "b1", "W2", "b2"):
        network.params[key] -= learning_rate*grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 4/10000 [02:38<109:50:54, 39.56s/it]


KeyboardInterrupt: 

In [ ]:
x_train.astype(np.array)

TypeError: Cannot interpret '<built-in function array>' as a data type